## Environment Preparation

In [0]:
!git clone --quiet https://github.com/ilopezfr/Traffic_Light_Detection.git tfd
  
!cp -r tfd/config config

!git clone --quiet https://github.com/tensorflow/models.git tf_models
  
!cp -r tf_models/research/object_detection object_detection/

!cp -r tf_models/research/slim slim/

!cp tfd/model_main.py object_detection/model_main.py

!rm -rf tfd

!rm -rf tf_models

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

!git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI

!make

%cd /content

!cp -r cocoapi/PythonAPI/pycocotools pycocotools/

!rm -rf cocoapi

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content:/content/slim

## Download datasets

In [0]:
# Install PyDrive
!pip install -U -q PyDrive

In [4]:
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# simulator data
# File url: https://drive.google.com/open?id=1-G066U5BUUNPvdinWrALX5XWcw7TioeY
simulator_file_id = '1-G066U5BUUNPvdinWrALX5XWcw7TioeY' 

data_dir = 'data'

%cd /content/Traffic_Light_Detection

if not os.path.isdir(data_dir):
  os.makedirs(data_dir)

downloaded = drive.CreateFile({'id': simulator_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'simulator.zip'))
#downloaded.GetContentFile(os.path.join(data_dir, 'mixed_train.record'))


/content/Traffic_Light_Detection


In [19]:
# carla_training data
# File url: https://drive.google.com/open?id=1pk7RZYcP57dxs-xmAHcdHseZ__ZtZWgB
carla_training_file_id = '1pk7RZYcP57dxs-xmAHcdHseZ__ZtZWgB' 

%cd /content/Traffic_Light_Detection
downloaded = drive.CreateFile({'id': carla_training_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'carla_training.zip'))

/content/Traffic_Light_Detection


In [20]:
# carla_testing data
# File url: https://drive.google.com/open?id=1A_lA1zdfRVDcaht2Z3z_iLkZHs7-QbbM
carla_testing_file_id = '1A_lA1zdfRVDcaht2Z3z_iLkZHs7-QbbM' 

%cd /content/Traffic_Light_Detection
downloaded = drive.CreateFile({'id': carla_testing_file_id})
downloaded.GetContentFile(os.path.join(data_dir, 'carla_testing.zip'))

/content/Traffic_Light_Detection


In [0]:
# Unzip files
%cd /content/Traffic_Light_Detection/data
!unzip simulator.zip
!unzip carla_training.zip 
!unzip carla_testing.zip 

## Convert images into TF Record files
The python program create_tf_record.py already splits the images into training and evaluation TF Record files. By default is 75% for training and 25% for evaluation.

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

For example for the simulator only:

In [0]:
%cd /content/Traffic_Light_Detection/
!python create_tf_record.py --data_dir=data/simulator \
                           --labels_dir=data/simulator/labels \
                           --labels_map_path=config/labels_map.pbtxt \
                           --output_path=data/simulator/simulator.record

For the mixed dataset

First we need to create a single folder with all the images from carla and the simulator, along with their respective labels. 

```
mixed
    L labels
         L img_01.xml
         L img_02.xml
         L ...
    L img_01.jpg
    L img_02.jpg
    L ...
    L mixed_train.record
    L mixed_eval.record

```

In [14]:
# Create the mixed dataset
%cd /content/Traffic_Light_Detection/data/
!mkdir mixed

# Copy Simulator files
%cp -a /content/Traffic_Light_Detection/data/simulator/. /content/Traffic_Light_Detection/data/mixed

# Copy Carla_1 (aka carla_training) files
%cp -a /content/Traffic_Light_Detection/data/carla_training/. /content/Traffic_Light_Detection/data/mixed

# Copy Carla_2 (aka carla_testing) files
%cp -a /content/Traffic_Light_Detection/data/carla_testing/. /content/Traffic_Light_Detection/data/mixed


/content/Traffic_Light_Detection/data
mkdir: cannot create directory ‘mixed’: File exists


In [23]:
%cd /content/Traffic_Light_Detection/
!python create_tf_record.py --data_dir=data/mixed \
                           --labels_dir=data/mixed/labels \
                           --labels_map_path=config/labels_map.pbtxt \
                           --output_path=data/mixed/mixed.record

/content/Traffic_Light_Detection
Total samples: 422
Converting:   0% 0/316 [00:00<?, ? images/s]/content/Traffic_Light_Detection/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:
Converting: 100% 316/316 [00:00<00:00, 967.13 images/s]
TF record file for training created with 316 samples: data/mixed/mixed_train.record
Converting: 100% 106/106 [00:00<00:00, 1020.48 images/s]
TF record file for validation created with 106 samples: data/mixed/mixed_eval.record
